In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # able to run multiple command in 1 line

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
folder = '/Users/amarindraardinova/Desktop/DataScience/Coursera/Project/Analize Data in Model Car Database/'
file_path = folder + 'join_table.csv' 

In [3]:
print(os.path.exists(file_path))

True


In [4]:
df = pd.read_csv(file_path,sep=';')


# EDA

In [5]:
#df = df[sorted(df.columns)]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12015 entries, 0 to 12014
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customerNumber          12015 non-null  int64  
 1   customerName            12015 non-null  object 
 2   contactLastName         12015 non-null  object 
 3   contactFirstName        12015 non-null  object 
 4   phone                   12015 non-null  object 
 5   addressLine1            12015 non-null  object 
 6   addressLine2            1827 non-null   object 
 7   city                    12015 non-null  object 
 8   state                   4750 non-null   object 
 9   postalCode              11545 non-null  object 
 10  country                 12015 non-null  object 
 11  salesRepEmployeeNumber  12015 non-null  int64  
 12  creditLimit             12015 non-null  float64
 13  orderNumber             12015 non-null  int64  
 14  orderDate               12015 non-null

## preprocessing : cleaning, rename, duplicate, missing values, feature engineering

In [7]:
# cleaning duplicates features :

columns_to_keep = [col for col in df.columns if not col.endswith('.1')]
df = df [columns_to_keep]

columns_to_keep = [col for col in df.columns if not col.endswith('.2')]
df = df [columns_to_keep]

df.drop(['htmlDescription','image'], axis=1, inplace=True)

In [8]:
df.duplicated().any()

np.False_

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12015 entries, 0 to 12014
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customerNumber          12015 non-null  int64  
 1   customerName            12015 non-null  object 
 2   contactLastName         12015 non-null  object 
 3   contactFirstName        12015 non-null  object 
 4   phone                   12015 non-null  object 
 5   addressLine1            12015 non-null  object 
 6   addressLine2            1827 non-null   object 
 7   city                    12015 non-null  object 
 8   state                   4750 non-null   object 
 9   postalCode              11545 non-null  object 
 10  country                 12015 non-null  object 
 11  salesRepEmployeeNumber  12015 non-null  int64  
 12  creditLimit             12015 non-null  float64
 13  orderNumber             12015 non-null  int64  
 14  orderDate               12015 non-null

### Missing Items

In [10]:
warehouse_cnt = df.groupby('warehouseName')[['productName', 'quantityInStock','quantityOrdered']].agg({'productName': ['count', 'nunique'], 'quantityInStock': 'sum', 'quantityOrdered': 'sum'}).reset_index()
warehouse_cnt.columns = ['warehouseName', 'totalrows', 'unique_product', 'USDValueInStock','TotalCustOrdered']
warehouse_cnt

,warehouseName,totalrows,unique_product,USDValueInStock,TotalCustOrdered
0,East,4341,37,24863942,153987
1,North,2363,25,12588144,84561
2,South,2761,23,9675107,97967
3,West,2550,24,13434712,88430


In [11]:
warehouse_cnt.totalrows.sum()

np.int64(12015)

In [12]:
productNameUnique_df = pd.Series(sorted(df.productName.unique().tolist())).to_frame(name='productName_df')
productNameUnique_df.sample()

,productName_df
88,2002 Chevy Corvette


In [13]:
productName_distinct = folder + 'productName_distinct.csv'
productName_distinct_sql = pd.read_csv(productName_distinct, sep=';')
productName_distinct_sql.sample()

,productName_mysql
47,1961 Chevrolet Impala


In [14]:
productName_distinct_sql[~productName_distinct_sql['productName_mysql'].isin(productNameUnique_df['productName_df'])]

,productName_mysql
76,1985 Toyota Supra


'1985 Toyota Supra' is the items not included in saved file of mySQL. We need to manually add this items information to df

In [15]:
# add '1985 Toyota Supra' to df
supra = pd.read_csv('1986Supra.csv', sep=';') 
df = pd.concat([df, supra], ignore_index=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12016 entries, 0 to 12015
Data columns (total 51 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customerNumber          12015 non-null  float64
 1   customerName            12015 non-null  object 
 2   contactLastName         12015 non-null  object 
 3   contactFirstName        12015 non-null  object 
 4   phone                   12015 non-null  object 
 5   addressLine1            12015 non-null  object 
 6   addressLine2            1827 non-null   object 
 7   city                    12015 non-null  object 
 8   state                   4750 non-null   object 
 9   postalCode              11545 non-null  object 
 10  country                 12015 non-null  object 
 11  salesRepEmployeeNumber  12015 non-null  float64
 12  creditLimit             12015 non-null  float64
 13  orderNumber             12015 non-null  float64
 14  orderDate               12015 non-null

In [17]:
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image
11071,141.0,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,1370.0,227600.0,10394.0,2005-03-15,2005-03-25,2005-03-19,Shipped,NaN,S24_4048,37.0,104.09,7.0,1992 Porsche Cayenne Turbo Silver,Classic Cars,1:24,Exoto Designs,"This replica features opening doors, superb de...",6582,b,69.78,118.28,Attention car enthusiasts: Make your wildest c...,East,67,MF629602,2004-08-16,20009.53,1370.0,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4.0,1102.0,Sales Rep,EMEA,NaN,NaN,NaN,NaN


In [18]:
df[df['productName'] == '1985 Toyota Supra']

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image
12015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S18_3233,NaN,NaN,NaN,1985 Toyota Supra,Classic Cars,1:18,Highway 66 Mini Classics,"This model features soft rubber tires, working...",7733,b,57.01,107.57,Attention car enthusiasts: Make your wildest c...,East,67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b,Classic Cars,NaN,NaN


## Total Investment (Purchase) & Expected Return

In [19]:
lp = df[['productName','productName','productLine','productScale',
         'productVendor','productDescription','quantityInStock',
         'quantityOrdered',	'priceEach','warehouseCode','buyPrice',
         'MSRP','warehouseName','warehousePctCap'
        ]].copy()

In [20]:
lp.sample()

,productName,productName,productLine,productScale,productVendor,productDescription,quantityInStock,quantityOrdered,priceEach,warehouseCode,buyPrice,MSRP,warehouseName,warehousePctCap
5349,1917 Grand Touring Sedan,1917 Grand Touring Sedan,Vintage Cars,1:18,Welly Diecast Productions,This 1:18 scale replica of the 1917 Grand Tour...,2724,49.0,137.7,c,86.7,170.0,West,50


In [21]:
lp['TotalInvestment']    = lp['quantityInStock'] * lp['buyPrice']
lp['ExpectedReturn']     = lp['quantityInStock'] * lp['MSRP']
lp['TotalSellingValue']  = lp['quantityOrdered'] * lp['priceEach']
lp['ProfitLost']         = lp['TotalSellingValue'] - lp['TotalInvestment']
lp.sample()

,productName,productName,productLine,productScale,productVendor,productDescription,quantityInStock,quantityOrdered,priceEach,warehouseCode,buyPrice,MSRP,warehouseName,warehousePctCap,TotalInvestment,ExpectedReturn,TotalSellingValue,ProfitLost
5776,1968 Ford Mustang,1968 Ford Mustang,Classic Cars,1:12,Autoart Studio Design,"Hood, doors and trunk all open to reveal highl...",68,44.0,188.73,b,95.34,194.57,East,67,6483.12,13230.76,8304.12,1821.0


In [22]:
lp.TotalInvestment.sum()
lp.ExpectedReturn.sum()

np.float64(3348745113.1500006)

np.float64(6181340333.71)

In [23]:
TotalInvestment         = lp['TotalInvestment'].sum()
ExpectedReturn          = lp['ExpectedReturn'].sum()
TotalSellingValue       = lp['TotalSellingValue'].sum()
ProfitLost              = lp['ProfitLost'].sum()
lp_pct                  = round(
                                (lp['TotalSellingValue'].sum() / lp['ExpectedReturn'].sum()) * 100
                                ,3)

formatted_TotalInvestment     = f"{TotalInvestment:,.2f}" 
formatted_ExpectedReturn      = f"{ExpectedReturn:,.2f}" 
formatted_TotalSellingValue   = f"{TotalSellingValue:,.2f}" 
formatted_ProfitLost          = f"{ProfitLost:,.2f}" 
status = "Lost" if ProfitLost < 0 else "Profit"


print(f"Total Investment           :  {formatted_TotalInvestment}")
print(f"Expected Return            :  {formatted_ExpectedReturn}")
print(f"Total Selling              :     {formatted_TotalSellingValue}")
print(f"Profit/Lost                : {formatted_ProfitLost}")
print(f"Profit/Lost Status         : {status}")
print(f"Selling Target Achievement : {lp_pct} %")


Total Investment           :  3,348,745,113.15
Expected Return            :  6,181,340,333.71
Total Selling              :     38,570,229.99
Profit/Lost                : -3,309,734,024.83
Profit/Lost Status         : Lost
Selling Target Achievement : 0.624 %


Here we can see company performance is far from satisfying, its target achievement is only 0.624%

## Payment Status

In [24]:
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image
4073,141.0,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,1370.0,227600.0,10212.0,2004-01-16,2004-01-24,2004-01-18,Shipped,NaN,S18_3232,40.0,155.79,11.0,1992 Ferrari 360 Spider red,Classic Cars,1:18,Unimax Art Galleries,"his replica features opening doors, superb det...",8347,b,77.9,169.34,Attention car enthusiasts: Make your wildest c...,East,67,DB583216,2004-11-01,36140.38,1370.0,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4.0,1102.0,Sales Rep,EMEA,NaN,NaN,NaN,NaN


In [25]:
df['orderDate']    = pd.to_datetime(df['orderDate'])
df['requiredDate'] = pd.to_datetime(df['requiredDate'])
df['shippedDate']  = pd.to_datetime(df['shippedDate'])
df['paymentDate']  = pd.to_datetime(df['paymentDate'])

In [26]:
df['orderDate'].min()
df['paymentDate'].max()

Timestamp('2003-01-06 00:00:00')

Timestamp('2005-06-09 00:00:00')

In [27]:
df['PaymentDuration'] = df['paymentDate'] - df['orderDate']

In [28]:
df['PaymentDuration'].max()

Timedelta('839 days 00:00:00')

In [29]:
df[df['productName'] == '1985 Toyota Supra']

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration
12015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,S18_3233,NaN,NaN,NaN,1985 Toyota Supra,Classic Cars,1:18,Highway 66 Mini Classics,"This model features soft rubber tires, working...",7733,b,57.01,107.57,Attention car enthusiasts: Make your wildest c...,East,67,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b,Classic Cars,NaN,NaN,NaT


In [30]:
df['PaymentDuration'].describe()

count                           12015
mean     -11 days +05:14:22.022471911
std       323 days 11:12:08.131203240
min               -826 days +00:00:00
25%               -217 days +00:00:00
50%                  14 days 00:00:00
75%                 166 days 00:00:00
max                 839 days 00:00:00
Name: PaymentDuration, dtype: object

In [31]:
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration
9955,141.0,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,1370.0,227600.0,10379.0,2005-02-10,2005-02-18,2005-02-11,Shipped,NaN,S24_1628,32.0,48.8,3.0,1966 Shelby Cobra 427 S/C,Classic Cars,1:24,Carousel DieCast Legends,This diecast model of the 1966 Shelby Cobra 42...,8197,b,29.18,50.31,Attention car enthusiasts: Make your wildest c...,East,67,JN722010,2003-02-25,40206.2,1370.0,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4.0,1102.0,Sales Rep,EMEA,NaN,NaN,NaN,NaN,-716 days


In [32]:
conditions = [
    (df['PaymentDuration'].dt.days < 0) & 
     (df['status'] == 'Shipped') & 
     (df['checkNumber'].notna()) & 
     (df['paymentDate'].notna()), 
    (df['PaymentDuration'].dt.days >= 0) & 
     (df['PaymentDuration'].dt.days < 100) & 
     (df['status'] == 'Shipped') & 
     (df['checkNumber'].notna()) & 
     (df['paymentDate'].notna()) 
]

choices = ['Paid', 'Due Date']

# conditional
df['purchasedstatus'] = np.select(conditions, choices, default='Non-performing Loan (NPL)')

In [33]:
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration,purchasedstatus
4938,141.0,Euro+ Shopping Channel,Freyre,Diego,(91) 555 94 44,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,1370.0,227600.0,10244.0,2004-04-29,2004-05-09,2004-05-04,Shipped,NaN,S18_2957,24.0,54.96,3.0,1934 Ford V8 Coupe,Vintage Cars,1:18,Min Lin Diecast,"Chrome Trim, Chrome Grille, Opening Hood, Open...",5649,c,34.35,62.46,Our Vintage Car models realistically portray a...,West,50,HJ32686,2004-01-30,59830.55,1370.0,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4.0,1102.0,Sales Rep,EMEA,NaN,NaN,NaN,NaN,-90 days,Paid


### Financial Status

In [34]:
df.purchasedstatus.value_counts()

purchasedstatus
Non-performing Loan (NPL)    4518
Paid                         4238
Due Date                     3260
Name: count, dtype: int64

In [35]:
df[df['purchasedstatus'] == 'Non-performing Loan (NPL)'].sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration,purchasedstatus
2753,324.0,"Stylish Desk Decors, Co.",Brown,Ann,(171) 555-0297,35 King George,NaN,London,NaN,WX3 6FW,UK,1501.0,77000.0,10175.0,2003-11-06,2003-11-14,2003-11-09,Shipped,NaN,S10_4962,33.0,119.67,9.0,1962 LanciaA Delta 16V,Classic Cars,1:10,Second Gear Diecast,Features include: Turnable front wheels; steer...,6791,b,103.42,147.74,Attention car enthusiasts: Make your wildest c...,East,67,DQ409197,2004-12-13,13671.82,1501.0,Bott,Larry,x2311,lbott@classicmodelcars.com,7.0,1102.0,Sales Rep,EMEA,NaN,NaN,NaN,NaN,403 days,Non-performing Loan (NPL)


In [36]:
df[df['checkNumber'].isna()].sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration,purchasedstatus
12015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,S18_3233,NaN,NaN,NaN,1985 Toyota Supra,Classic Cars,1:18,Highway 66 Mini Classics,"This model features soft rubber tires, working...",7733,b,57.01,107.57,Attention car enthusiasts: Make your wildest c...,East,67,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b,Classic Cars,NaN,NaN,NaT,Non-performing Loan (NPL)


In [37]:
df['TotalInvestment']    = df['quantityInStock'] * df['buyPrice']
df['ExpectedReturn']     = df['quantityInStock'] * df['MSRP']
df['TotalSellingValue']  = df['quantityOrdered'] * df['priceEach']
df['ProfitLost']         = df['TotalSellingValue'] - df['TotalInvestment']
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration,purchasedstatus,TotalInvestment,ExpectedReturn,TotalSellingValue,ProfitLost
6542,166.0,Handji Gifts& Co,Victorino,Wendy,+65 224 1555,106 Linden Road Sandown,2nd Floor,Singapore,NaN,069045,Singapore,1612.0,97900.0,10288.0,2004-09-01,2004-09-11,2004-09-05,Shipped,NaN,S18_4933,23.0,57.02,7.0,1957 Ford Thunderbird,Classic Cars,1:18,Studio M Art Models,"This 1:18 scale precision die-cast replica, wi...",3209,b,34.21,71.27,Attention car enthusiasts: Make your wildest c...,East,67,BQ327613,2004-09-16,38785.48,1612.0,Marsh,Peter,x102,pmarsh@classicmodelcars.com,6.0,1088.0,Sales Rep,APAC,NaN,NaN,NaN,NaN,15 days,Due Date,109779.89,228705.43,1311.46,-108468.43


In [38]:
npl = df[df['purchasedstatus'] == 'Non-performing Loan (NPL)']
pl = df[df['purchasedstatus'] == 'Paid']
dd = df[df['purchasedstatus'] == 'Due Date']

# step 1
npl_profit_lost = npl['ProfitLost'].sum()
pl_profit_lost  = pl['ProfitLost'].sum() 
dd_profit_lost  = dd['ProfitLost'].sum()

# round
formatted_npl_profit_lost = f"{npl_profit_lost:,.2f}" 
formatted_pl_profit_lost   = f"{pl_profit_lost:,.2f}" 
formatted_dd_profit_lost   = f"{dd_profit_lost:,.2f}" 

# result
print("Total Profit Lost (NPL)     :", formatted_npl_profit_lost)
print("Total Profit Lost (Paid)    :", formatted_pl_profit_lost)
print("Total Profit Lost (Due Date):  ", formatted_dd_profit_lost)

Total Profit Lost (NPL)     : -1,218,550,715.04
Total Profit Lost (Paid)    : -1,186,899,746.31
Total Profit Lost (Due Date):   -904,283,563.48


In [39]:
df.purchasedstatus.value_counts()

purchasedstatus
Non-performing Loan (NPL)    4518
Paid                         4238
Due Date                     3260
Name: count, dtype: int64

Here i misunderstand the data, company lost was not (mostly) caused by customers behavior, i than need to Analyze Customer Payment Behaviour

### Customer Payment Analysis

In [40]:
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration,purchasedstatus,TotalInvestment,ExpectedReturn,TotalSellingValue,ProfitLost
6136,112.0,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,NaN,Las Vegas,NV,83030,USA,1166.0,71800.0,10278.0,2004-08-06,2004-08-16,2004-08-09,Shipped,NaN,S18_1984,29.0,118.07,5.0,1995 Honda Civic,Classic Cars,1:18,Min Lin Diecast,"This model features, opening hood, opening doo...",9772,b,93.89,142.25,Attention car enthusiasts: Make your wildest c...,East,67,BO864823,2004-12-17,14191.12,1166.0,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1.0,1143.0,Sales Rep,NaN,NaN,NaN,NaN,NaN,133 days,Non-performing Loan (NPL),917493.08,1390067.0,3424.03,-914069.05


In [41]:
df['custPaid']         = df['amount']
df['custInvoice']      = df['quantityOrdered'] * df['priceEach']
df['custDebtCredit']  = df['custPaid'] - df['custInvoice']

In [42]:
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration,purchasedstatus,TotalInvestment,ExpectedReturn,TotalSellingValue,ProfitLost,custPaid,custInvoice,custDebtCredit
4615,124.0,Mini Gifts Distributors Ltd.,Nelson,Susan,4155551450,5677 Strong St.,NaN,San Rafael,CA,97562,USA,1165.0,210500.0,10229.0,2004-03-11,2004-03-20,2004-03-12,Shipped,NaN,S24_2840,33.0,34.65,2.0,1958 Chevy Corvette Limited Edition,Classic Cars,1:24,Carousel DieCast Legends,The operating parts of this 1958 Chevy Corvett...,2542,b,15.91,35.36,Attention car enthusiasts: Make your wildest c...,East,67,HI366474,2004-12-27,47142.7,1165.0,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1.0,1143.0,Sales Rep,NaN,NaN,NaN,NaN,NaN,291 days,Non-performing Loan (NPL),40443.22,89885.12,1143.45,-39299.77,47142.7,1143.45,45999.25


In [43]:
npl = df[df['purchasedstatus'] == 'Non-performing Loan (NPL)']
pl = df[df['purchasedstatus'] == 'Paid']
dd = df[df['purchasedstatus'] == 'Due Date']

# step 1
npl_co_lost = npl['custDebtCredit'].sum()
pl_co_lost  = pl['custDebtCredit'].sum() 
dd_co_lost  = dd['custDebtCredit'].sum()

# round
formatted_npl_co_lost  = f"{npl_co_lost:,.2f}" 
formatted_pl_co_lost   = f"{pl_co_lost:,.2f}" 
formatted_dd_co_lost   = f"{dd_co_lost:,.2f}" 
formatted_total_co_lost= f"{npl_co_lost+pl_co_lost+dd_co_lost:,.2f}" 

# result
print(f"Total Investment                    : ", formatted_TotalInvestment)
print(f"Total Selling Performance           :    ", formatted_TotalSellingValue)
print()
print("Total Customer Debt/Credit (NPL)     :  ", formatted_npl_co_lost)
print("Total Customer Debt/Credit (Paid)    :  ", formatted_pl_co_lost)
print("Total Customer Debt/Credit (Due Date):  ", formatted_dd_co_lost)
print("Total Debt/Credit                    :  ", formatted_total_co_lost)

Total Investment                    :  3,348,745,113.15
Total Selling Performance           :     38,570,229.99

Total Customer Debt/Credit (NPL)     :   190,316,677.24
Total Customer Debt/Credit (Paid)    :   156,311,145.27
Total Customer Debt/Credit (Due Date):   139,477,862.52
Total Debt/Credit                    :   486,105,685.03


In [44]:
df.sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration,purchasedstatus,TotalInvestment,ExpectedReturn,TotalSellingValue,ProfitLost,custPaid,custInvoice,custDebtCredit
7529,462.0,FunGiftIdeas.com,Benitez,Violeta,5085552555,1785 First Street,NaN,New Bedford,MA,50553,USA,1216.0,85800.0,10321.0,2004-11-04,2004-11-12,2004-11-07,Shipped,NaN,S24_3856,26.0,137.62,13.0,1956 Porsche 356A Coupe,Classic Cars,1:18,Classic Metal Creations,Features include: Turnable front wheels; steer...,6600,b,98.3,140.43,Attention car enthusiasts: Make your wildest c...,East,67,GC60330,2003-11-08,9977.85,1216.0,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2.0,1143.0,Sales Rep,NaN,NaN,NaN,NaN,NaN,-362 days,Paid,648780.0,926838.0,3578.12,-645201.88,9977.85,3578.12,6399.73


In [60]:
df[df['custDebtCredit'] < 0].shape[0]

268

In [63]:
df[df['custDebtCredit'] < 0].sample()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,orderNumber,orderDate,requiredDate,shippedDate,status,comments,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,warehouseCode,buyPrice,MSRP,textDescription,warehouseName,warehousePctCap,checkNumber,paymentDate,amount,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,territory,warehouseCode.1,productLine.1,htmlDescription,image,PaymentDuration,purchasedstatus,TotalInvestment,ExpectedReturn,TotalSellingValue,ProfitLost,custPaid,custInvoice,custDebtCredit
5419,398.0,"Tokyo Collectables, Ltd",Shimamura,Akiko,+81 3 3584 0555,2-2-8 Roppongi,NaN,Minato-ku,Tokyo,106-0032,Japan,1621.0,94400.0,10258.0,2004-06-15,2004-06-25,2004-06-23,Shipped,NaN,S10_1949,32.0,177.87,6.0,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,b,98.58,214.3,Attention car enthusiasts: Make your wildest c...,East,67,JPMR4544,2005-05-18,615.45,1621.0,Nishi,Mami,x101,mnishi@classicmodelcars.com,5.0,1056.0,Sales Rep,Japan,NaN,NaN,NaN,NaN,337 days,Non-performing Loan (NPL),720126.9,1565461.5,5691.84,-714435.06,615.45,5691.84,-5076.39


In [66]:
custDebtCredit_minus = df[df['custDebtCredit'] < 0]

In [72]:
custDebtCredit_minus['custDebtCredit'].sum()

np.float64(-524646.73)

In [81]:
custDebtCredit_minus_df = custDebtCredit_minus['country'].value_counts().reset_index()
custDebtCredit_minus_df['count'].sum()

np.int64(268)

In [80]:
country_df = df['country'].value_counts().reset_index()
country_df['count'].sum()

np.int64(12015)

In [55]:
# Customer Performance to Payment by Nation
customersNationPayment = df.groupby('country')[['custPaid', 'custInvoice', 'custDebtCredit']].sum().reset_index()

# sorting 'custDebtCredit'
customersNationPayment.sort_values('custDebtCredit', ascending=False, inplace=True)

# avoid scientific notation in 'custDebtCredit'
customersNationPayment[['custPaid', 'custInvoice', 'custDebtCredit']] = customersNationPayment[['custPaid', 'custInvoice', 'custDebtCredit']].applymap(lambda x: f'{x:,.2f}')

customersNationPayment.reset_index(drop=True, inplace=True)
customersNationPayment

,country,custPaid,custInvoice,custDebtCredit
0,Spain,"191,923,032.08","11,407,323.68","180,515,708.40"
1,USA,"175,064,862.42","12,187,006.48","162,877,855.94"
2,France,"29,316,168.56","3,028,447.14","26,287,721.42"
3,Australia,"22,312,940.91","1,949,501.04","20,363,439.87"
4,New Zealand,"16,016,145.94","1,400,398.97","14,615,746.97"
5,UK,"13,658,649.09","1,102,861.70","12,555,787.39"
6,Italy,"11,977,684.75","848,763.31","11,128,921.44"
7,Singapore,"10,513,934.81","948,244.37","9,565,690.44"
8,Finland,"9,045,892.80","885,448.05","8,160,444.75"
9,Denmark,"6,295,638.60","696,160.08","5,599,478.52"


In [58]:
f"{df['custDebtCredit'].sum():,.2f}"

'486,105,685.03'

Customers payment performace are good enough leads by USA & Spain, so the lost was not caused by customers payment, company needs to expand business to capture more market. Company can scale customer with hugh credit to purchase more items to maximize profit.